<a href="https://colab.research.google.com/github/jmanning21/31005_MachineLearning_Python/blob/master/Machine_Learning_Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Assignment 2: Practical Machine Learning Project
####Affnan Amir (13528841) and Julia Manning (12875795)

###Introduction
The algorithm chosen for this project is the ID3 Decision Tree building algorithm. In using this alrogithm, the data  

* Input = 
* Output = 

###Exploration

Challenges:
 - over-fitting the data

###Methodology

In [0]:
# Importing the required packages
import os
import numpy as np
import pandas as pd
from sklearn import tree, metrics
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [0]:
# Importing the dataset from a url
url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"

# Read the csv and convert to a dataframe
d = pd.read_csv(url)
df = pd.DataFrame(data=d)

# See the first six rows of the dataset
df.head(6)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q


In [0]:
# Cleaning some of the attributes: replace null values with median values
# Replacing NA values in age with the median age of each sex
median_values = df.groupby('Sex')['Age'].transform('median')
df['Age'].fillna(median_values, inplace=True)
df.head(6) # see the first 6 rows to see if the NA value was replaced in Ln 6

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,29.0,0,0,330877,8.4583,NaN,Q


In [0]:
# Check the information about the dataset
df.info()
# There are no missing values in the dataset

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            891 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [0]:
# Identify the target variable
df['Survived'],class_names = pd.factorize(df['Survived'])

print(class_names)
print(df['Survived'].unique())

# The algorithm requires the target variable to be in integer format, which it already is. 
# Therefore no data type changes need to be made.

Int64Index([0, 1], dtype='int64')
[0 1]


In [0]:
# Adding and removing specific columns which may help with the accuracy of the accuracy.
# Add a Child.Adult column based on the age of the individual
df['Child.Adult'] = 'Adult'

# Child = Age < 13 and Adult = Age >= 13 
df['Child.Adult'][df['Age'] < 13.0] = 'Child'

# Remove 'Name' column as it does not provide any relevant information to the dataset, as all the values are unique
del df['Name']
# Remove 'Cabin' as there are too many NA values and the remaining values are all unique, make it impossible to determine a suitable replacement for the NA values
del df['Cabin']

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [0]:
# Identifying the predictor variables and encode any string variables to their equivalent integer formats
df['Sex'],_ = pd.factorize(df['Sex'])
df['Embarked'],_ = pd.factorize(df['Embarked'])
df['Child.Adult'],_ = pd.factorize(df['Child.Adult'])
df['Ticket'],_ = pd.factorize(df['Ticket'])

df.head(10)

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Child.Adult
0,1,0,3,0,22.0,1,0,0,7.2500,0,0
1,2,1,1,1,38.0,1,0,1,71.2833,1,0
2,3,1,3,1,26.0,0,0,2,7.9250,0,0
3,4,1,1,1,35.0,1,0,3,53.1000,0,0
4,5,0,3,0,35.0,0,0,4,8.0500,0,0
5,6,0,3,0,29.0,0,0,5,8.4583,2,0
6,7,0,1,0,54.0,0,0,6,51.8625,0,0
7,8,0,3,0,2.0,3,1,7,21.0750,0,1
8,9,1,3,1,27.0,0,2,8,11.1333,0,0
9,10,1,2,1,14.0,1,0,9,30.0708,1,0


In [0]:
df.info()

# Re-ordering the columns to put the 'Survived' column last
df = df[['PassengerId','Pclass','Sex','Age','SibSp','Parch','Ticket','Fare','Embarked','Child.Adult','Survived']]
df.head(6)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Sex            891 non-null int64
Age            891 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null int64
Fare           891 non-null float64
Embarked       891 non-null int64
Child.Adult    891 non-null int64
dtypes: float64(2), int64(9)
memory usage: 76.6 KB


,PassengerId,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Child.Adult,Survived
0,1,3,0,22.0,1,0,0,7.2500,0,0,0
1,2,1,1,38.0,1,0,1,71.2833,1,0,1
2,3,3,1,26.0,0,0,2,7.9250,0,0,1
3,4,1,1,35.0,1,0,3,53.1000,0,0,1
4,5,3,0,35.0,0,0,4,8.0500,0,0,0
5,6,3,0,29.0,0,0,5,8.4583,2,0,0


In [0]:

# Selecting the predictor feature and target variable
X = df.iloc[:,:-1]
y = df.iloc[:,-1]

In [0]:
# split data randomly into 60% training and 40% test
train_data_split = 0.6 #@param {type:"number", min:0.01, max:0.99}
test_data_split = 1 - train_data_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_data_split, random_state=0)

0.6
0.4


###Evaluation
Improvements for future renditions: 
- When producing the ages for null values in the dataset, the ages could have been grouped by class and gender of the individual, rather than just using gender. This may produce a more accurate number. The age range for females is: 0.75-63, giving a range difference of 62.25, whilst the age range for males is: 0.42-80, giving a age range of 79.58. The age range is also specified further when Pclass is added.
- Use Gini Idex instead of Entropy?

###Conclusion

###Ethical Justification

- potential misuses of the technique
- utilitarian view?

###Video Pitch
*Attached to the final document*